<a href="https://colab.research.google.com/github/siyandaV-0/llm-playground/blob/main/Code%20Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code Generator

The requirement: use a Frontier and Open Source model to generate high performance C++ code from Python code

# Important Note

In this project we use Frontier models (GPT-4o and Claude-3.5-Sonnet, Gemini) and some Open-source models (CODE-QWEN1.5-Instruct & Deepseek Instruct). We deploy 1 of each to a Gradio Application.

In [1]:
!pip3 install -q --no-cache-dir torch==2.5.1 torchvision==0.20.1 torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 128.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 147.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 148.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 151.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 172.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 172.0 MB/s eta 0:00:00


In [2]:
!pip3 install -U -qq --no-cache-dir openai python-dotenv anthropic gradio transformers bitsandbytes huggingface_hub requests sentencepiece accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 144.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 MB 177.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 233.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 126.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 272.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 179.7 MB/s eta 0:00:00


In [3]:
!pip install -U -q "google-generativeai>=0.7.2" # Install the Python SDK

In [7]:

# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
from google import genai
from google.genai import types
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
from google.colab import userdata

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [8]:
# environment

# load_dotenv()
# os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
# os.environ['HF_TKEN'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
# os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')

# Colab Environment Variable setup
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [9]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [10]:
0# initialize
# NOTE - option to use ultra-low cost models by uncommenting last 2 lines

openai = OpenAI()
claude = anthropic.Anthropic()
client = genai.Client(api_key=userdata.get("GEMINI_API_KEY"))
OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"
CODE_QWEN_MODEL = "Qwen/CodeQwen1.5-7B-Chat"
DEEPSEEK = "deepseek-ai/deepseek-coder-6.7b-instruct"

In [11]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an M2 MacBook. "
system_message += "Respond only with C++ code; use comments sparingly and DO NOT include the user prompts and DO NOT provide any code explanation!"
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [12]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [13]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [14]:
import re

def write_output(cpp):
    # Extract only the content inside the ```cpp block
    match = re.search(r"```cpp\s*(.*?)\s*```", cpp, re.DOTALL)
    if match:
        code = match.group(1).strip()  # Extract the C++ code
    else:
        code = cpp.strip()  # Fallback: If no ```cpp``` block is found, use the original input

    with open("optimized.cpp", "w") as f:
        f.write(code)

In [15]:
def optimize_gpt(python):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [16]:
def optimize_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)

In [18]:
def optimize_gemini(python):
  response = client.models.generate_content_stream(
      model="gemini-2.0-flash",
      config=types.GenerateContentConfig(system_instruction=system_message),
      contents=[user_prompt_for(python)]
  )
  reply= ""
  for chunk in response:
      reply += chunk.text
      print(chunk.text, end="")
  write_output(reply)


In [19]:
# Quantization Config - this allows us to load the model into memory and use less memory

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    # bnb_8bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [20]:
def optimize_qwen(python):
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(CODE_QWEN_MODEL, use_fast=True)
    tokenizer.pad_token = tokenizer.eos_token

    # Prepare inputs
    inputs = tokenizer.apply_chat_template(messages_for(python), return_tensors="pt").to("cuda")

    # Load model
    model = AutoModelForCausalLM.from_pretrained(CODE_QWEN_MODEL, quantization_config=quant_config, torch_dtype="auto", device_map="auto")

    # Generate response
    outputs = model.generate(inputs, max_new_tokens=512)
    reply = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Print and save output
    print(reply)
    write_output(reply)

In [21]:
def optimize_deepseek(python):
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(DEEPSEEK, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token


    inputs = tokenizer.apply_chat_template(
            # conversation=[{"role": "user", "content": user_prompt_for(python)}], # Pass the conversation object here
            conversation=messages_for(python), # Pass the conversation object here
            add_generation_prompt=True,
            return_tensors="pt"
        ).to("cuda")

    # Load model
    model = AutoModelForCausalLM.from_pretrained(DEEPSEEK, quantization_config=quant_config, torch_dtype="auto", device_map="auto")

    # Generate response
    outputs = model.generate(inputs, max_new_tokens=512)
    reply = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Print and save output
    print(reply)
    write_output(reply)

In [22]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [23]:
exec(pi)

Result: 3.141592658589
Execution Time: 15.334281 seconds


In [24]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(int iterations, double param1, double param2) {
    double result = 1.0;
    for (int i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100000000, 4.0, 1.0) * 4.0;
    auto end_time = std::chrono::high_resolution_clock::now();
    
    std::cout << std::fixed << std::setprecision(12) << "Result: " << result << std::endl;
    std::chrono::duration<double> execution_time = end_time - start_time;
    std::cout << std::fixed << std::setprecision(6) << "Execution Time: " << execution_time.count() << " seconds" << std::endl;

    return 0;
}
```

# Compiling C++ and executing

This next cell contains the command to compile a C++ file on colab
.  
It compiles the file `optimized.cpp` into an executable called `optimized`  
Then it runs the program called `optimized`

You can google (or ask ChatGPT!) for how to do this on your platform, then replace the lines below.
If you're not comfortable with this step, you can skip it for sure - I'll show you exactly how it performs on my Mac.

In [25]:
# Compile C++ and run the executable for gpt results
!clang++ -O3 -std=c++17 -march=native -o optimized optimized.cpp
!./optimized

Result: 3.141592658589
Execution Time: 0.257884 seconds


In [ ]:
# optimize_claude(pi)

In [26]:

optimize_gemini(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(long long iterations, int param1, int param2) {
    double result = 1.0;
    for (long long i = 1; i <= iterations; ++i) {
        double j = static_cast<double>(i * param1) - param2;
        result -= (1.0 / j);
        j = static_cast<double>(i * param1) + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100000000LL, 4, 1) * 4;
    auto end_time = std::chrono::high_resolution_clock::now();

    std::cout << std::fixed << std::setprecision(12) << "Result: " << result << std::endl;
    std::cout << std::fixed << std::setprecision(6) << "Execution Time: " << std::chrono::duration<double>(end_time - start_time).count() << " seconds" << std::endl;

    return 0;
}
```

In [27]:
# Compile C++ and run the executable for gemini results
!clang++ -O3 -std=c++17 -march=native -o optimized optimized.cpp
!./optimized

Result: 3.141592658589
Execution Time: 0.253754 seconds


In [28]:
optimize_deepseek(pi)

tokenizer_config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


You are an assistant that reimplements Python code in high performance C++ for an M2 MacBook. Respond only with C++ code; use comments sparingly and DO NOT include the user prompts and DO NOT provide any code explanation!The C++ response needs to produce an identical output in the fastest possible time.### Instruction:
Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. Respond only with C++ code; do not explain your work other than a few comments. Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.


import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {re

In [29]:
# Compile C++ and run the executable for deepseek
!clang++ -O3 -std=c++17 -march=native -o optimized optimized.cpp
!./optimized

Result: 3.141592658589
Execution Time: 0.256796466000 seconds


In [30]:
optimize_qwen(pi)

tokenizer_config.json:   0%|          | 0.00/972 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.46M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/31.7k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.71G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

system
You are an assistant that reimplements Python code in high performance C++ for an M2 MacBook. Respond only with C++ code; use comments sparingly and DO NOT include the user prompts and DO NOT provide any code explanation!The C++ response needs to produce an identical output in the fastest possible time.
user
Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. Respond only with C++ code; do not explain your work other than a few comments. Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.


import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result

In [31]:
# Compile C++ and run the executable for code qwen results
!clang++ -O3 -std=c++17 -march=native -o optimized optimized.cpp
!./optimized

Result: 3.141592658589
Execution Time: 0.263707 seconds


In [32]:
python_hard = """
def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value

def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [33]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 50.690906 seconds


In [34]:
optimize_gpt(python_hard)

```cpp
#include <iostream>
#include <iomanip>
#include <vector>
#include <chrono>

class LCG {
public:
    LCG(unsigned long long seed, unsigned long long a = 1664525, unsigned long long c = 1013904223, unsigned long long m = 1ull << 32)
        : a(a), c(c), m(m), value(seed) {}

    unsigned long long next() {
        value = (a * value + c) % m;
        return value;
    }

private:
    unsigned long long a, c, m, value;
};

long long max_subarray_sum(int n, unsigned long long seed, int min_val, int max_val) {
    LCG lcg(seed);
    std::vector<int> random_numbers(n);
    for (int &num : random_numbers) {
        num = lcg.next() % (max_val - min_val + 1) + min_val;
    }

    long long max_sum = LLONG_MIN;
    for (int i = 0; i < n; ++i) {
        long long current_sum = 0;
        for (int j = i; j < n; ++j) {
            current_sum += random_numbers[j];
            if (current_sum > max_sum) {
                max_sum = current_sum;
            }
        }
    }
    return max_su

In [35]:
# Compile C++ and run the executable for gpt results
!clang++ -O3 -std=c++17 -march=native -o optimized optimized.cpp
!./optimized

optimized.cpp:27:25: error: use of undeclared identifier 'LLONG_MIN'
    long long max_sum = LLONG_MIN;
                        ^
1 error generated.
Result: 3.141592658589
Execution Time: 0.254118 seconds


In [ ]:
# optimize_claude(python_hard)

In [36]:
optimize_gemini(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <limits>
#include <chrono>
#include <iomanip>

using namespace std;

// LCG function
uint32_t lcg(uint32_t& value, uint32_t a, uint32_t c, uint32_t m) {
    value = (static_cast<uint64_t>(a) * value + c) % m;
    return value;
}

// Max subarray sum function
long long max_subarray_sum(int n, uint32_t seed, int min_val, int max_val) {
    vector<int> random_numbers(n);
    uint32_t value = seed;
    for (int i = 0; i < n; ++i) {
        random_numbers[i] = lcg(value, 1664525, 1013904223, 0xFFFFFFFF) % (max_val - min_val + 1) + min_val;
    }

    long long max_sum = numeric_limits<long long>::min();
    for (int i = 0; i < n; ++i) {
        long long current_sum = 0;
        for (int j = i; j < n; ++j) {
            current_sum += random_numbers[j];
            if (current_sum > max_sum) {
                max_sum = current_sum;
            }
        }
    }
    return max_sum;
}

// Total max subarray sum function
long long total_max

In [37]:
# Compile C++ and run the executable for gemini results
!clang++ -O3 -std=c++17 -march=native -o optimized optimized.cpp
!./optimized

Total Maximum Subarray Sum (20 runs): 3480
Execution Time: 0.646753 seconds


In [38]:
optimize_deepseek(python_hard)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


You are an assistant that reimplements Python code in high performance C++ for an M2 MacBook. Respond only with C++ code; use comments sparingly and DO NOT include the user prompts and DO NOT provide any code explanation!The C++ response needs to produce an identical output in the fastest possible time.### Instruction:
Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. Respond only with C++ code; do not explain your work other than a few comments. Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.


def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value

def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n

In [39]:
# Compile C++ and run the executable for deepseek results
!clang++ -O3 -std=c++17 -march=native -o optimized optimized.cpp
!./optimized

optimized.cpp:1:1: error: unknown type name 'You'
You are an assistant that reimplements Python code in high performance C++ for an M2 MacBook...
^
optimized.cpp:1:8: error: expected ';' after top level declarator
You are an assistant that reimplements Python code in high performance C++ for an M2 MacBook...
       ^
       ;
optimized.cpp:1:123: error: unknown type name 'use'
  ...code in high performance C++ for an M2 MacBook. Respond only with C++ code; use comments spa...
                                                                                 ^
optimized.cpp:1:135: error: expected ';' after top level declarator
  ...performance C++ for an M2 MacBook. Respond only with C++ code; use comments sparingly and D...
                                                                                ^
                                                                                ;
optimized.cpp:2:152: error: expected unqualified-id
  ...produces identical output in the least time. Re

In [41]:
optimize_qwen(python_hard)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

system
You are an assistant that reimplements Python code in high performance C++ for an M2 MacBook. Respond only with C++ code; use comments sparingly and DO NOT include the user prompts and DO NOT provide any code explanation!The C++ response needs to produce an identical output in the fastest possible time.
user
Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. Respond only with C++ code; do not explain your work other than a few comments. Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.


def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value

def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
 

In [42]:
def stream_gpt(python):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)

    capturing = False  # Flag to track if we're inside the ```cpp block
    reply = ""

    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment

        if not capturing:
            # Look for the start of the C++ block
            match = re.search(r"```cpp\s*(.*)", reply, re.DOTALL)
            if match:
                capturing = True
                reply = match.group(1)  # Keep only the code after ```cpp

        if capturing:
            # Remove trailing triple backticks if they appear
            reply = re.sub(r"\s*```.*", "", reply).strip()

            yield reply  # Stream the cleaned-up C++ code

In [43]:
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )

    capturing = False  # Flag to track if we are inside the ```cpp block
    reply = ""

    with result as stream:
        for text in stream.text_stream:
            reply += text

            if not capturing:
                # Look for the start of the C++ block
                match = re.search(r"```cpp\s*(.*)", reply, re.DOTALL)
                if match:
                    capturing = True
                    reply = match.group(1)  # Keep only the code after ```cpp

            if capturing:
                # Remove trailing triple backticks if they appear
                cleaned_reply = re.sub(r"\s*```.*", "", reply).strip()

                yield cleaned_reply  # Stream the cleaned-up C++ code

In [44]:
def stream_gemini(python):
    response = client.models.generate_content_stream(
        model="gemini-2.0-flash",
        config=types.GenerateContentConfig(system_instruction=system_message),
        contents=[user_prompt_for(python)]
    )

    capturing = False  # Flag to track if we are inside the ```cpp block
    reply = ""

    for chunk in response:
        fragment = chunk.text or ""
        reply += fragment

        if not capturing:
            # Look for the start of the C++ block
            match = re.search(r"```cpp\s*(.*)", reply, re.DOTALL)
            if match:
                capturing = True
                reply = match.group(1)  # Keep only the code after ```cpp

        if capturing:
            # Remove trailing triple backticks if they appear
            cleaned_reply = re.sub(r"\s*```.*", "", reply).strip()

            yield cleaned_reply  # Stream the cleaned-up C++ code

In [45]:
def stream_qwen(python):
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(CODE_QWEN_MODEL, use_fast=True)
    tokenizer.pad_token = tokenizer.eos_token

    # Prepare inputs
    inputs = tokenizer.apply_chat_template(messages_for(python), return_tensors="pt").to("cuda")

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        CODE_QWEN_MODEL,
        quantization_config=quant_config,
        torch_dtype="auto",
        device_map="auto"
    )

    # Generate response stream
    capturing = False  # Track if we are inside ```cpp block
    reply = ""

    for outputs in model.generate(inputs, max_new_tokens=512, do_sample=True):
        fragment = tokenizer.decode(outputs, skip_special_tokens=True)
        reply += fragment

        if not capturing:
            # Detect the start of C++ code block
            match = re.search(r"```cpp\s*(.*)", reply, re.DOTALL)
            if match:
                capturing = True
                reply = match.group(1)  # Keep only the code after ```cpp

        if capturing:
            # Remove trailing triple backticks if they appear
            cleaned_reply = re.sub(r"\s*```.*", "", reply).strip()

            yield cleaned_reply  # Stream the cleaned-up C++ code

In [46]:
def stream_deepseek(python):
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(DEEPSEEK, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token

    # Prepare inputs
    inputs = tokenizer.apply_chat_template(
        conversation=messages_for(python),  # Pass conversation messages
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        DEEPSEEK,
        quantization_config=quant_config,
        torch_dtype="auto",
        device_map="auto"
    )

    # Generate response
    capturing = False  # Flag to track if we are inside the ```cpp block
    reply = ""

    for outputs in model.generate(inputs, max_new_tokens=512, do_sample=True):
        fragment = tokenizer.decode(outputs, skip_special_tokens=True)
        reply += fragment

        if not capturing:
            # Detect the start of C++ code block
            match = re.search(r"```cpp\s*(.*)", reply, re.DOTALL)
            if match:
                capturing = True
                reply = match.group(1)  # Keep only the code after ```cpp

        if capturing:
            # Remove trailing triple backticks if they appear
            cleaned_reply = re.sub(r"\s*```.*", "", reply).strip()

            yield cleaned_reply  # Stream the cleaned-up C++ code

In [48]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    # elif model=="Claude":
    #     result = stream_claude(python)
    elif model=="Gemini":
        result = stream_gemini(python)
    elif model=="Qwen":
        result = stream_qwen(python)
    elif model=="Deepseek":
        result = stream_deepseek(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far

In [49]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=python_hard)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Gemini", "Qwen", "Deepseek"], label="Select model", value="GPT")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ed06f756ad542804b7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [50]:
def execute_python(code):
        try:
            output = io.StringIO()
            sys.stdout = output
            exec(code)
        finally:
            sys.stdout = sys.__stdout__
        return output.getvalue()

In [51]:
# You'll need to change the code in the try block to compile the C++ code for your platform
# I pasted this into Claude's chat UI with a request for it to give me a version for an Intel PC,
# and it responded with something that looks perfect - you can try a similar approach for your platform.

# M1 Mac version to compile and execute optimized C++ code:
# !clang++ -O3 -std=c++17 -march=native -o optimized optimized.cpp
def execute_cpp(code):
        write_output(code)
        try:
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=native", "-o", "optimized", "optimized.cpp"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [53]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [54]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Gemini", "Qwen", "Deepseek"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4443caba404f166988.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [55]:
# Clean up

# del inputs, outputs, model
torch.cuda.empty_cache()